In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout
import time
import psutil

df1 = pd.read_csv('sms-call-internet-mi-2013-11-04.csv')
df2 = pd.read_csv('sms-call-internet-mi-2013-11-05.csv')
df3 = pd.read_csv('sms-call-internet-mi-2013-11-06.csv')
df4 = pd.read_csv('sms-call-internet-mi-2013-11-07.csv')
df5 = pd.read_csv('sms-call-internet-mi-2013-11-08.csv')
df6 = pd.read_csv('sms-call-internet-mi-2013-11-09.csv')
df7 = pd.read_csv('sms-call-internet-mi-2013-11-10.csv')

dfs = [df1, df2, df3, df4, df5, df6, df7]
df = pd.concat(dfs, ignore_index=True)
df = df.drop(columns=['datetime'])
df.dropna(inplace=True)


cols = ["smsin", "smsout", "callin", "callout", "internet"]
df["total_activity"] = df[cols].sum(axis=1)


def assign_activity(total_activity):
    if total_activity <= 20:
        return 1
    elif total_activity <= 40:
        return 2
    elif total_activity <= 60:
        return 3
    elif total_activity <= 80:
        return 4
    elif total_activity <= 100:
        return 5
    else:
        return 6

df['activity_number'] = df['total_activity'].apply(assign_activity)
df['activity_number'] = df['activity_number'] - 1

X = df[["smsin", "smsout", "callin", "callout", "internet"]]
y = df[["activity_number"]]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


model = Sequential()
model.add(SimpleRNN(64, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(16, return_sequences=False))
model.add(Dense(6, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



model.fit(X_train, y_train, epochs=10, batch_size=10000)


Epoch 1/10
606/606 [==============================] - 20s 32ms/step - loss: 0.4573 - accuracy: 0.8519
Epoch 2/10
606/606 [==============================] - 100s 165ms/step - loss: 0.0834 - accuracy: 0.9827
Epoch 3/10
606/606 [==============================] - 19s 32ms/step - loss: 0.0493 - accuracy: 0.9875
Epoch 4/10
606/606 [==============================] - 21s 34ms/step - loss: 0.0372 - accuracy: 0.9894
Epoch 5/10
606/606 [==============================] - 21s 34ms/step - loss: 0.0303 - accuracy: 0.9909
Epoch 6/10
606/606 [==============================] - 21s 35ms/step - loss: 0.0263 - accuracy: 0.9917
Epoch 7/10
606/606 [==============================] - 21s 35ms/step - loss: 0.0238 - accuracy: 0.9919
Epoch 8/10
606/606 [==============================] - 21s 34ms/step - loss: 0.0209 - accuracy: 0.9930
Epoch 9/10
606/606 [==============================] - 22s 36ms/step - loss: 0.0200 - accuracy: 0.9928
Epoch 10/10
606/606 [==============================] - 21s 35ms/step - loss: 0.0